In [4]:
import math
import numpy as np
import pandas as pd
import os
import glob

from pykrx import stock
from pykrx import bond

import time
from time import sleep
from datetime import datetime
from datetime import timedelta

from pyarrow import csv
import pyarrow as pa
import pyarrow.parquet as pq

from ta.trend import MACD
from ta.momentum import StochasticOscillator

import psycopg2 as pg2
from sqlalchemy import create_engine

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go

from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import storage


import warnings
warnings.filterwarnings('ignore')

# 경로 변경
os.chdir('/home/shjj08choi4/finance_mlops')


# 서비스 계정 키 JSON 파일 경로
key_path = glob.glob("key_value/*.json")[0]

# Credentials 객체 생성
credentials = service_account.Credentials.from_service_account_file(key_path)

# 빅쿼리 정보
project_id = 'owenchoi-404302'
dataset_id = 'finance_mlops'


# BigQuery 클라이언트 객체 생성
client = bigquery.Client(credentials = credentials, 
                         project = credentials.project_id)


# GCP 클라이언트 객체 생성
storage_client = storage.Client(credentials = credentials,
                         project = credentials.project_id)
bucket_name = 'finance-mlops-proj'    # 서비스 계정 생성한 bucket 이름 입력


now = datetime.now()
# now = now + timedelta(days=-2)
today_date1 = now.strftime('%Y%m%d')
today_date2 = now.strftime('%Y-%m-%d')
today_date_time_csv = now.strftime("%Y%m%d_%H%M")

today_date1 = '2024010'
today_date2 = '2023-01-07'

today_date1 = 'reset'

In [7]:
sql = f"""
select 
  `date`,
  open, 
  high, 
  low, 
  close, 
  volume,
  price_change_percentage,
  `{project_id}.{dataset_id}.kor_stock_ohlcv`.ticker,
  corp_name, 
  market 
from `{project_id}.{dataset_id}.kor_stock_ohlcv`
left join  `{project_id}.{dataset_id}.kor_ticker_list`
on `{project_id}.{dataset_id}.kor_stock_ohlcv`.ticker = `{project_id}.{dataset_id}.kor_stock_ohlcv`.ticker
order by date asc
"""

### 비트코인 전처리

In [8]:
# 비트코인
sql = f"""
select 
*
from `owenchoi-404302.finance_mlops.bitcoin`
order by date asc
"""

In [32]:
# 데이터 조회 쿼리 실행 결과
query_job = client.query(sql)

# 데이터프레임 변환
ohlcv_df_raw = query_job.to_dataframe()

ohlcv_df_raw = ohlcv_df_raw.fillna(0)
# ticker_list = ohlcv_df_raw['ticker'].unique()

In [33]:
ohlcv_df_raw

,date,open,high,low,close,adj_close,volume
0,2016-01-01 00:00:00+00:00,5.060929e+05,5.125672e+05,5.022874e+05,5.101036e+05,5.101036e+05,42607754105
1,2016-01-02 00:00:00+00:00,5.104418e+05,5.121330e+05,5.072097e+05,5.090512e+05,5.090512e+05,35346951870
2,2016-01-03 00:00:00+00:00,5.092157e+05,5.094095e+05,4.989786e+05,5.051554e+05,5.051554e+05,46559806550
3,2016-01-04 00:00:00+00:00,5.052142e+05,5.172751e+05,5.059330e+05,5.146464e+05,5.146464e+05,45723198025
4,2016-01-05 00:00:00+00:00,5.146202e+05,5.148096e+05,5.100812e+05,5.139287e+05,5.139287e+05,41073608576
...,...,...,...,...,...,...,...
2921,2023-12-31 00:00:00+00:00,5.448904e+07,5.512642e+07,5.379578e+07,5.457337e+07,5.457337e+07,20730507552967
2922,2024-01-01 00:00:00+00:00,5.456716e+07,5.548477e+07,5.436800e+07,5.473514e+07,5.473514e+07,21235430222208
2923,2024-01-02 00:00:00+00:00,5.475463e+07,5.657283e+07,5.464855e+07,5.656245e+07,5.656245e+07,23598325673712
2924,2024-01-03 00:00:00+00:00,5.658782e+07,6.008276e+07,5.657477e+07,5.893540e+07,5.893540e+07,51564611389277


In [12]:
# df_raw = ohlcv_df_raw[ohlcv_df_raw['ticker'] == ticker_nm].reset_index(drop = True)
df_raw = ohlcv_df_raw
######################################################################
# 보조지표
######################################################################

# 이동평균선
df_raw['MA5'] = df_raw['close'].rolling(window=5).mean()
df_raw['MA20'] = df_raw['close'].rolling(window=20).mean()
df_raw['MA60'] = df_raw['close'].rolling(window=60).mean()
df_raw['MA120'] = df_raw['close'].rolling(window=120).mean()

# 볼린저밴드
std = df_raw['close'].rolling(20).std(ddof=0)

df_raw['upper'] = df_raw['MA20'] + 2 * std
df_raw['lower'] = df_raw['MA20'] - 2 * std

# MACD
# MACD
macd = MACD(close=df_raw['close'],
            window_slow=26,
            window_fast=12,
            window_sign=9)


df_raw['MACD_DIFF'] = macd.macd_diff()
df_raw['MACD'] = macd.macd()
df_raw['MACD_Signal'] = macd.macd_signal()

# RSI
df_raw['변화량'] = df_raw['close'] - df_raw['close'].shift(1)
df_raw['변화량'] = df_raw['변화량'].astype('float64')
df_raw['상승폭'] = np.where(df_raw['변화량']>=0, df_raw['변화량'], 0)
df_raw['하락폭'] = np.where(df_raw['변화량'] <0, df_raw['변화량'].abs(), 0)

# welles moving average
df_raw['AU'] = df_raw['상승폭'].ewm(alpha=1/14, min_periods=14).mean()
df_raw['AD'] = df_raw['하락폭'].ewm(alpha=1/14, min_periods=14).mean()
df_raw['RSI'] = df_raw['AU'] / (df_raw['AU'] + df_raw['AD']) * 100

df_raw['MA5-20'] = df_raw['MA5'] - df_raw['MA20']
df_raw['MA20-60'] = df_raw['MA20'] - df_raw['MA60']
df_raw['MA60-120'] = df_raw['MA60'] - df_raw['MA120']


######################################################################
# 보조지표 분석
######################################################################
# df_raw_anal = df_raw[['date','ticker', 'corp_name','market', 'close']]
df_raw_anal = df_raw[['date','close']]

# 골든크로스
# 골든 크로스 5-20
# 음수에서 양수로 바뀌는 모든 인덱스 찾기
idx_5_20_gold_cross = [idx for idx in range(len(df_raw)) if df_raw["MA5-20"].iloc[idx] > 0 and df_raw["MA5-20"].iloc[idx - 1] <= 0]

# 데드 크로스 5-20
# 양수에서 음수로 바뀌는 모든 인덱스 찾기
idx_5_20_dead_cross = [idx for idx in range(len(df_raw)) if df_raw["MA5-20"].iloc[idx] < 0 and df_raw["MA5-20"].iloc[idx - 1] >= 0]

# 골든 크로스 20-60
# 음수에서 양수로 바뀌는 모든 인덱스 찾기
idx_20_60_gold_cross = [idx for idx in range(len(df_raw)) if df_raw["MA20-60"].iloc[idx] > 0 and df_raw["MA20-60"].iloc[idx - 1] <= 0]

# 골든 크로스 20-60
# 음수에서 양수로 바뀌는 모든 인덱스 찾기
idx_20_60_dead_cross = [idx for idx in range(len(df_raw)) if df_raw["MA20-60"].iloc[idx] < 0 and df_raw["MA20-60"].iloc[idx - 1] >= 0]


df_raw_anal.loc[:, '5_20_cross'] = '-'
df_raw_anal.loc[idx_5_20_gold_cross,'5_20_cross'] = '골든크로스(매수)'
df_raw_anal.loc[idx_5_20_dead_cross,'5_20_cross'] = '데드크로스(매도)'

df_raw_anal.loc[:, '20_60_cross'] = '-'
df_raw_anal.loc[idx_20_60_gold_cross,'20_60_cross'] = '골든크로스(매수)'
df_raw_anal.loc[idx_20_60_dead_cross,'20_60_cross'] = '데드크로스(매도)'


# 정배열 역배열
ascending_sq  = (df_raw['MA5-20'] > 0) & \
(df_raw['MA20-60'] > 0) & \
(df_raw['MA60-120'] > 0)

descending_sq  = (df_raw['MA5-20'] < 0) & \
(df_raw['MA20-60'] < 0) & \
(df_raw['MA60-120'] < 0)

df_raw_anal.loc[:,'array'] = '-'
df_raw_anal.loc[ascending_sq,'array'] = '정배열(매수)'
df_raw_anal.loc[descending_sq,'array'] = '역배열(매도)'


# 볼린저밴드
df_raw['close'] = df_raw['close'].astype('float64')
down_reg_sq = df_raw['upper'] - df_raw['close']
top_reg_sq  = df_raw['lower'] - df_raw['close']

down_reg = [idx for idx in range(1,len(df_raw)) if down_reg_sq[idx] > 0 and down_reg_sq[idx-1] <= 0]
top_reg = [idx for idx in range(1,len(df_raw)) if top_reg_sq[idx] < 0 and top_reg_sq[idx-1] >= 0]

df_raw_anal.loc[:,'Bollinger_band'] = '-'
df_raw_anal.loc[down_reg,'Bollinger_band'] = '하향회귀(매도)'
df_raw_anal.loc[top_reg,'Bollinger_band'] = '상향회귀(매수)'


# MACD
signal_down_cross = [idx for idx in range(1,len(df_raw)) if df_raw['MACD_DIFF'][idx] < 0 and df_raw['MACD_DIFF'][idx-1] >= 0]
signal_top_corss = [idx for idx in range(1,len(df_raw)) if df_raw['MACD_DIFF'][idx] > 0 and df_raw['MACD_DIFF'][idx-1] <= 0]

df_raw_anal.loc[:,'MACD'] = '-'
df_raw_anal.loc[signal_down_cross,'MACD'] = '하향돌파(매도)'
df_raw_anal.loc[signal_top_corss,'MACD'] = '상향돌파(매수)'

# RSI
down_reg = [idx for idx in range(1,len(df_raw)) if df_raw['RSI'][idx] > 70 and df_raw['RSI'][idx-1] <= 70]
top_reg = [idx for idx in range(1,len(df_raw)) if df_raw['RSI'][idx] < 30 and df_raw['RSI'][idx-1] >= 30]


df_raw_anal.loc[:,'RSI'] = '-'
df_raw_anal.loc[down_reg,'RSI'] = 'RSI 상단 하향돌파(매도)'
df_raw_anal.loc[top_reg,'RSI'] = 'RSI 하단 상향 돌파(매수)'


# df_raw_total = pd.concat([df_raw_total, df_raw])
# df_raw_anal_total = pd.concat([df_raw_anal_total, df_raw_anal])

In [22]:

df_raw_total_2 = df_raw[df_raw['date'] > '2023-01-01'].reset_index(drop = True)
df_raw_anal_total_2 = df_raw_anal[df_raw_anal['date'] > '2023-01-01'].reset_index(drop = True)

In [23]:
file_name = 'bitcoin'
if not os.path.exists(f'data_crawler/cleaning/{file_name}'):
    os.makedirs(f'data_crawler/cleaning/{file_name}')

In [27]:
table_from_pandas = pa.Table.from_pandas(df_raw_total_2,preserve_index = False)
pq.write_table(table_from_pandas, f'data_crawler/cleaning/{file_name}/{file_name}_cleaning.parquet')

table_from_pandas = pa.Table.from_pandas(df_raw_anal_total_2,preserve_index = False)
pq.write_table(table_from_pandas, f'data_crawler/cleaning/{file_name}/{file_name}_anal_cleaning.parquet')


# Google Storage 적재
source_file_name = f'data_crawler/cleaning/{file_name}/{file_name}_cleaning.parquet'    # GCP에 업로드할 파일 절대경로
destination_blob_name = f'data_crawler/cleaning/{file_name}/{file_name}_cleaning.parquet'    # 업로드할 파일을 GCP에 저장할 때의 이름
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)


# Google Storage 적재
source_file_name = f'data_crawler/cleaning/{file_name}/{file_name}_anal_cleaning.parquet'    # GCP에 업로드할 파일 절대경로
destination_blob_name = f'data_crawler/cleaning/{file_name}/{file_name}_anal_cleaning.parquet'    # 업로드할 파일을 GCP에 저장할 때의 이름
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)

In [16]:
df_raw_anal.tail(3)

,date,close,5_20_cross,20_60_cross,array,Bollinger_band,MACD,RSI
2923,2024-01-02 00:00:00+00:00,56562448.0,-,-,-,-,-,-
2924,2024-01-03 00:00:00+00:00,58935404.0,골든크로스(매수),-,정배열(매수),-,-,-
2925,2024-01-04 00:00:00+00:00,56141780.0,-,-,정배열(매수),하향회귀(매도),-,-


In [17]:
df_raw.tail()

,date,open,high,low,close,adj_close,volume,MA5,MA20,MA60,...,MACD_Signal,변화량,상승폭,하락폭,AU,AD,RSI,MA5-20,MA20-60,MA60-120
2921,2023-12-31 00:00:00+00:00,54489036.0,55126424.0,53795780.0,54573372.0,54573372.0,20730507552967,55028684.0,55393868.8,5.159899e+07,...,1.082244e+06,74432.0,74432.0,0.0,434219.690664,440441.742132,49.644317,-365184.8,3.794877e+06,6.929690e+06
2922,2024-01-01 00:00:00+00:00,54567156.0,55484768.0,54368000.0,54735140.0,54735140.0,21235430222208,54984172.0,55419051.0,5.171493e+07,...,9.749453e+05,161768.0,161768.0,0.0,414758.855616,408981.617694,50.350671,-434879.0,3.704124e+06,6.873722e+06
2923,2024-01-02 00:00:00+00:00,54754632.0,56572832.0,54648552.0,56562448.0,56562448.0,23598325673712,55038447.2,55529826.0,5.188066e+07,...,9.014949e+05,1827308.0,1827308.0,0.0,515655.223072,379768.645002,57.587835,-491378.8,3.649169e+06,6.853597e+06
2924,2024-01-03 00:00:00+00:00,56587816.0,60082764.0,56574768.0,58935404.0,58935404.0,51564611389277,55861060.8,55703650.2,5.210516e+07,...,8.889151e+05,2372956.0,2372956.0,0.0,648319.564282,352642.313216,64.769656,157410.6,3.598488e+06,6.871065e+06
2925,2024-01-04 00:00:00+00:00,58940164.0,59517664.0,53559256.0,56141780.0,56141780.0,60719981782131,56189628.8,55740126.2,5.227547e+07,...,8.685506e+05,-2793624.0,0.0,2793624.0,602011.023976,526998.147986,53.322067,449502.6,3.464653e+06,6.859700e+06
